In [16]:
import torch
import ray
from ray import tune
from ray.air import session
from sklearn.model_selection import train_test_split
import yaml
import argparse
import shutil
from collections import namedtuple
import os
import datetime


# Custom Libraries
from utils.data_generator import DataGenerator
from utils.agent import TrainModel
from utils.helper import to_python_native, gen_experiment_name, set_seed, save_model_state
from utils.plotter import alignment_progress_over_iterations, plot_initial_final_accuracies
from utils.pennylane.model import Qkernel

In [17]:
# === Backend Configuration ===
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.xpu.is_available():
    device = torch.device("xpu")
else:
    device = torch.device("cpu")

set_seed(42)

In [18]:
data_generator = DataGenerator(
        dataset_name= 'iris',
        file_path=None,
    )

training_data, training_labels, testing_data, testing_labels = data_generator.generate_dataset()
training_data = torch.tensor(training_data.to_numpy(), dtype=torch.float32, requires_grad=True)
testing_data = torch.tensor(testing_data.to_numpy(), dtype=torch.float32, requires_grad=True)
training_labels = torch.tensor(training_labels.to_numpy(), dtype=torch.int)
testing_labels = torch.tensor(testing_labels.to_numpy(), dtype=torch.int)

In [19]:
kernel = Qkernel(
        device='lightning.qubit',
        n_qubits=4,
        trainable=True,
        input_scaling=True,
        data_reuploading=True,
        ansatz='embedding_paper',
        ansatz_layers=3
    )

In [20]:
agent = TrainModel(
        kernel=kernel,
        training_data=training_data,
        training_labels=training_labels,
        testing_data=testing_data,
        testing_labels=testing_labels,
        optimizer='gd',
        lr= 0.1,
        mclr=0.01,
        cclr=0.01,
        epochs=500,
        train_method='ccka',
        target_accuracy=0.95,
        get_alignment_every=10,
        validate_every_epoch=1,
        base_path='./',
        lambda_kao=0.001,
        lambda_co=0.001,
        clusters=4
    )

Epochs:  50


In [21]:
before_metrics = agent.evaluate(testing_data, testing_labels, 'before')
before_metrics

{'alignment': tensor(2.7980, grad_fn=<SqueezeBackward0>),
 'executions': None,
 'training_accuracy': 0.96,
 'testing_accuracy': 0.9866666666666667,
 'f1_score': 0.9866603624901497,
 'alignment_arr': [[], []],
 'loss_arr': [],
 'validation_accuracy_arr': []}

In [22]:
agent.fit_multiclass(training_data, training_labels)

Started Training
------------------------------------------------------------------
Epoch: 0 — Alignment per main centroid
Centroid 0 (label=0): Alignment = -0.8014624118804932
Centroid 1 (label=1): Alignment = -0.21288105845451355
Centroid 2 (label=2): Alignment = -0.43874871730804443
------------------------------------------------------------------
------------------------------------------------------------------
Epoch: 1 — Alignment per main centroid
Centroid 0 (label=0): Alignment = -0.7896039485931396
Centroid 1 (label=1): Alignment = -0.21670088171958923
Centroid 2 (label=2): Alignment = -0.3790322542190552
------------------------------------------------------------------
------------------------------------------------------------------
Epoch: 2 — Alignment per main centroid
Centroid 0 (label=0): Alignment = -0.7636924982070923
Centroid 1 (label=1): Alignment = -0.19821147620677948
Centroid 2 (label=2): Alignment = -0.32627707719802856
----------------------------------------

(Qkernel(),
 [Parameter containing:
  tensor([[1.2712, 0.3658, 1.3034, 0.5713],
          [1.4484, 1.2594, 0.0663, 0.8611],
          [0.9213, 2.6027, 0.4531, 1.0570]], requires_grad=True),
  Parameter containing:
  tensor([[ 2.9346,  1.9459,  0.0571,  1.8929],
          [-1.5099,  0.2755, -1.7183,  1.3920],
          [ 2.0516, -1.7074,  2.0229,  0.4356]], requires_grad=True),
  Parameter containing:
  tensor([[ 2.1246,  0.8600,  1.3919,  0.0103],
          [ 2.3950,  0.0687, -2.1493,  0.1087],
          [-1.0723, -0.2729, -0.9452,  1.5438]], requires_grad=True)],
 ParameterList(
     (0): Parameter containing: [torch.float32 of size 4]
     (1): Parameter containing: [torch.float32 of size 4]
     (2): Parameter containing: [torch.float32 of size 4]
 ),
 ParameterList(
     (0): Parameter containing: [torch.float32 of size 4]
     (1): Parameter containing: [torch.float32 of size 4]
     (2): Parameter containing: [torch.float32 of size 4]
     (3): Parameter containing: [torch.float3

In [23]:
after_metrics = agent.evaluate(testing_data, testing_labels, 'after')
after_metrics

{'alignment': tensor(2.7540, grad_fn=<SqueezeBackward0>),
 'executions': 18000,
 'training_accuracy': 0.9466666666666667,
 'testing_accuracy': 0.76,
 'f1_score': 0.7616374269005847,
 'alignment_arr': [[], []],
 'loss_arr': [],
 'validation_accuracy_arr': []}